In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline


# df1 = pd.read_csv('./reviews.csv')
# print('----------------------------')
# print(df1.head())

# df2 = pd.read_csv('./calendar.csv')
# print('----------------------------')
# print(df2.head())

# is there any relationship between host response time and score rating??
# pd.set_option('display.max_columns', None)
df = pd.read_csv('./listings.csv')
# df = pd.read_csv('./survey_results_public.csv')
# print(df[['review_scores_rating',
#        'review_scores_accuracy', 'review_scores_cleanliness',
#        'review_scores_checkin', 'review_scores_communication',
#        'review_scores_location', 'review_scores_value']])
print(df[['city']])
print('----------------------------')
# print(df3[['host_is_superhost', 'review_scores_rating']].dropna())
print('----------------------------')
# print(df3[['host_response_time', 'review_scores_rating']].dropna())

         city
0     Seattle
1     Seattle
2     Seattle
3     Seattle
4     Seattle
...       ...
3813  Seattle
3814  Seattle
3815  Seattle
3816  Seattle
3817  Seattle

[3818 rows x 1 columns]
----------------------------
----------------------------


In [2]:
def clean_data(df):
    '''
    INPUT
    df - pandas dataframe 
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector
    
    This function cleans df using the following steps to produce X and y:
    1. Drop all the rows with no salaries
    2. Create X as all the columns that are not the Salary column
    3. Create y as the Salary column
    4. Drop the Salary, Respondent, and the ExpectedSalary columns from X
    5. For each numeric variable in X, fill the column with the mean value of the column.
    6. Create dummy columns for all the categorical variables in X, drop the original columns
    '''
    # Drop rows with missing salary values
    df = df.dropna(subset=['review_scores_rating'], axis=0)
    y = df['review_scores_rating']
    
    #Drop respondent and expected salary columns
    df = df.drop(['id', 'host_id', 'city', 'state', 'license','zipcode','monthly_price','host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'experiences_offered',
                  'host_neighbourhood','neighborhood_overview','neighbourhood_group_cleansed', 'notes', 'transit', 'summary','space', 'description', 'number_of_reviews',
                  'street', 'market', 'smart_location', 'country_code', 'country', 'host_thumbnail_url','host_picture_url','thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'first_review', 
                  'last_review', 'review_scores_rating','review_scores_accuracy', 'review_scores_cleanliness','review_scores_checkin', 
                  'review_scores_communication', 'review_scores_location', 'review_scores_value','reviews_per_month', 'latitude', 'amenities', 
                  'longitude', 'calendar_last_scraped', 'square_feet','weekly_price'], axis=1)
    
    print(df.head())
    # Fill numeric columns with the mean
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)
        
    # Dummy the categorical variables
    cat_vars = df.select_dtypes(include=['object']).copy().columns
    for var in  cat_vars:
        # for each cat add dummy var, drop original column
        df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)
        
    X = df
    return X, y

#Use the function to create X and y
X, y = clean_data(df) 

   host_response_time host_response_rate host_acceptance_rate  \
0  within a few hours                96%                 100%   
1      within an hour                98%                 100%   
2  within a few hours                67%                 100%   
4      within an hour               100%                  NaN   
5                 NaN                NaN                  NaN   

  host_is_superhost  host_listings_count  host_total_listings_count  \
0                 f                  3.0                        3.0   
1                 t                  6.0                        6.0   
2                 f                  2.0                        2.0   
4                 f                  2.0                        2.0   
5                 f                  1.0                        1.0   

                                  host_verifications host_has_profile_pic  \
0               ['email', 'phone', 'reviews', 'kba']                    t   
1  ['email', 'phone', 'faceb

In [5]:
# Split data into training and test data, and fit a linear model
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=.30, random_state=42)
lm_2_model = LinearRegression(normalize=True)
pd.set_option('display.max_columns', None)
X_train.to_csv('X_train', sep='\t')
# print(X_train.head())

# If our model works, it should just fit our model to the data. Otherwise, it will let us know.
try:
    lm_2_model.fit(X_train, y_train)
#     y_test_preds = lm_2_model.predict(X_test)# Predictions here
#     r2_test = r2_score(y_test, y_test_preds)
except Exception as e:
    print(e)

/Users/georgekorovesis/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
